# Fine Tuning the pre-trained BERT on Downstream Task: 'Question - Answering'

In the question-answering task, BERT has to return the text span that contains the answers from the context/paragraphs.

Our model has to understand the starting and ending index of the text span containing the answers. How? We have to calcuate the conditional probability of each token of the feature of being starting and ending token of the answers. Our model has pick the text span containing the answers using the maximum of logits of each token and each feature (question,context pair).

![](https://blog.floydhub.com/content/images/size/w2000/2018/10/tra.jpeg)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from random import sample
import os,gc,re
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")

### Loading the data

In [ ]:
def data_loader():
    train   = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/train.csv')
    train.drop_duplicates(keep='first')
    test   = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/test.csv')
    submission = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/sample_submission.csv')
    return train,test,submission

train,test,_ = data_loader()

train.info()

In [ ]:
train.language.value_counts()

### Preprocessing the data

In [ ]:
# function to get the starting index of character of answer text in the context and answer_text
def get_answers(x):
    start = x[0]
    text = x[1]
    return {
        'answer_start': [start],
        'text': [text]
    }

In [ ]:
train['answers'] = train[['answer_start', 'answer_text']].apply(get_answers, axis=1)
pd.set_option('display.max_colwidth',None)
train.head(1)

In [ ]:
test.head(1)

### Importing the Transformer Model from Huggingface 

In [ ]:
#!pip install -U --no-build-isolation --no-deps ../input/transformers-master/ -qq

In [ ]:
!pip install transformers -q

In [ ]:
import transformers
print(transformers.__version__)

In [ ]:
from transformers import AutoTokenizer
  
model_checkpoint = "salti/bert-base-multilingual-cased-finetuned-squad"


## added these models to input from kaggle
#'../input/xlm-roberta-squad2/deepset/xlm-roberta-large-squad2' --> mean jaccard 0.68
#'../input/xlm-roberta-squad2/deepset/xlm-roberta-base-squad2' --> mean jaccard 0.56
# alon-albalak/xlm-roberta-large-xquad -0.58
# 'setu4993/LaBSE' --> 0.4055
# "bert-large-uncased-whole-word-masking-finetuned-squad" --> poor results
# The following  models also tried:
# neuralspace-reverie/indic-transformers-hi-bert
# neuralspace-reverie/indic-transformers-bn-distilbert
# ai4bharat/indic-bert

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

One specific thing for the preprocessing in question answering is how to deal with very long documents. We usually truncate them in other tasks, when they are longer than the model maximum sentence length, but here, removing part of the the context might result in losing the answer we are looking for. To deal with this, we will allow one (long) example in our dataset to give several input features, each of length shorter than the maximum length of the model (or the one we set as a hyper-parameter). Also, just in case the answer lies at the point we split a long context, we allow some overlap between the features we generate controlled by the hyper-parameter 'doc_stride'

In [ ]:
!pip install datasets -qq # disable it to submit the notebook as it needs internet. Kaggle competitions does not use internet.

In [ ]:
!pip uninstall fsspec -qq -y
!pip install --no-index --find-links ../input/hf-datasets/wheels datasets -qq # added to input from kaggle

In [ ]:
import torch
from datasets import Dataset

In [ ]:
train = train.sample(frac=1, axis=None,random_state=41).reset_index(drop=True)
df_train = train[:-128].reset_index(drop=True)
df_valid = train[-128:].reset_index(drop=True)

train_dataset = Dataset.from_pandas(df_train)
valid_dataset = Dataset.from_pandas(df_valid)
df_train.shape,df_valid.shape

In [ ]:
print(valid_dataset[0]) # This is the standard format of dataset to be tokenized by the HF trasnformenr models

In [ ]:
pad_on_right = tokenizer.padding_side == "right"

### Refer the following link to get know the details of the generic function to prepare the data to feed to the model
link: https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb

In the case of impossible answers (the answer is in another feature given by an example with a long context), we set the cls index for both the start and end position. We could also simply discard those examples from the training set if the flag allow_impossible_answers is False.

In [ ]:
max_length = 512 # The maximum length of a feature (question and context)
doc_stride = 256 # The authorized overlap between two part of the context when splitting it is needed.

In [ ]:
def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
tokenized_train = train_dataset.map(prepare_train_features, batched=True, remove_columns=train_dataset.column_names)
tokenized_valid = valid_dataset.map(prepare_train_features, batched=True, remove_columns=valid_dataset.column_names)

In [ ]:
gc.collect()

In [ ]:
print(tokenized_train[0])

## Instantiating Trainer

To instantiate a Trainer, we will need to define three more things. The most important is the TrainingArguments, which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional.

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

The warning is telling us we are throwing away some weights (the vocab_transform and vocab_layer_norm layers) and randomly initializing some other (the pre_classifier and classifier layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

In [ ]:
# determine the device we will be using for training
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("[INFO] training using {}".format(torch.cuda.get_device_name(0)))
#print('There are %d GPU(s) available.' % torch.cuda.device_count())

### To avoid “CUDA out of memory” in PyTorch

In [ ]:
gc.collect()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
%env WANDB_DISABLED=True

In [ ]:
model_name = model_checkpoint.split("/")[-1]
epoch= 1
batch_size = 4
weight_decay= 3e-7
warmup_steps = 500

args = TrainingArguments(
    output_dir='./results2',
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    warmup_ratio= 0.1,
    warmup_steps = warmup_steps,
    weight_decay = weight_decay,
    gradient_accumulation_steps = 2,
    fp16= True,
    fp16_opt_level= "O1",
    seed= 0,
    load_best_model_at_end=True
)

we will need a data collator that will batch our processed examples together, here the default one will work:

In [ ]:
from transformers import default_data_collator

data_collator = default_data_collator

In [ ]:
trainer = Trainer(
    model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset =tokenized_valid,
    data_collator=data_collator,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

In [ ]:
#trainer.save_model("chaii-xlm-roberta-large-squad2-model")

In [ ]:
#trainer.evaluate()

## Evaluation
Evaluating the model will require a bit more work, as we will need to map the predictions of the model back to parts of the context. The model itself predicts logits for the start and end position of the 'answers': if we take a batch from our validation datalaoder.

The most obvious thing to predict an answer for each feature is to take the **index for the maximum of the start logits** as a start position and **the index of the maximum of the end logits** as an end position.

This will work great in a lot of cases, but what if this prediction gives us something impossible: the start position could be greater than the end position, or point to a span of text in the question instead of the answer. In that case, we might want to look at the second best prediction to see if it gives a possible answer and select that instead.

## Valid Features

we can sort the valid_answers according to their score and only keep the best one. 

The only point left is how to check a given span is inside the context (and not the question) and how to get back the text inside. To do this, we need to add two things to our validation features:

the ID of the example that generated the feature (since each example can generate several features, as seen before);
the offset mapping that will give us a map from token indices to character positions in the context.

In [ ]:
def prepare_validation_features(examples):
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
valid_features = valid_dataset.map(prepare_validation_features,batched=True,remove_columns=valid_dataset.column_names)

Since one example can give several features, we will need to gather together all the answers in all the features generated by a given example, then pick the best one. The following code builds a map from example index to its corresponding features indices:

In [ ]:
import collections

examples = valid_dataset
features = valid_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)

for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

The code above only keeps answers that are inside the context, we will need to grab the score for the impossible answer (which has start and end indices corresponding to the index of the **CLS token**). 

We predict the impossible answer when the score is greater than the score of the best non-impossible answer. All combined together, this gives us the following post-processing function:

In [ ]:
gc.collect()

In [ ]:
from tqdm.auto import tqdm
squad_v2 = False

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 24):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer
    
    return predictions

In [ ]:
valid_feats = valid_features.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])
raw_predictions = trainer.predict(valid_feats)

In [ ]:
final_predictions = postprocess_qa_predictions(valid_dataset, valid_features, raw_predictions.predictions)

In [ ]:
valid_preds = [{"id": ex["id"], "answer": ex["answers"]['text'][0]} for ex in valid_dataset]
pred_valid_df= pd.DataFrame(valid_preds)
pred_valid_df.head(1)

In [ ]:
def jaccard(row): 
    str1 = row[0]
    str2 = row[1]
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
pred_valid_df['prediction'] = pred_valid_df['id'].apply(lambda r: final_predictions[r])
pred_valid_df['jaccard'] = pred_valid_df[['answer', 'prediction']].apply(jaccard, axis=1)
pred_valid_df.head()

In [ ]:
round(pred_valid_df.jaccard.mean(),4)

## Predictions of Test data

In [ ]:
_,_,sub=data_loader()

In [ ]:
test_dataset = Dataset.from_pandas(test)
test_features = test_dataset.map(
    prepare_validation_features,
    batched=True,
    remove_columns=test_dataset.column_names
)

In [ ]:
print(test_features[0])

In [ ]:
test_feats = test_features.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])
test_feats

In [ ]:
raw_test_predictions = trainer.predict(test_feats)

In [ ]:
test_features.set_format(type=test_features.format["type"], columns=list(test_features.features.keys()))
final_test_predictions = postprocess_qa_predictions(test_dataset, test_features, raw_test_predictions.predictions)

In [ ]:
sub['PredictionString'] = sub['id'].apply(lambda x: final_test_predictions[x])

In [ ]:
sub['Correct Answers'] = ['येलन','28 नवम्बर 2007','१२ मार्च १८२४','13','சுவாமிநாதன் மற்றும் வர்கீஸ் குரியன்']# As I get it manually.
sub['jaccard score']   = sub[['PredictionString', 'Correct Answers']].apply(jaccard, axis=1)
print(sub.head())

In [ ]:
sub.to_csv('submission.csv',index=False)